# 预备任务 #
## 1.安装tensorflow ##
安装Tensorflow的通用方法参见[tensorflow官网](https://www.tensorflow.org/install)。下面具体介绍下我自己的安装过程。
### 1.1 安装Anaconda ###
这个在之前使用Jupyter Notebook的时候就安装好了的，具体安装过程参见[Anaconda官网](https://www.anaconda.com/distribution/)。

### 1.2 创建新的虚拟环境 ##
为了方便管理创建一个基于python3.6的新工作环境，命名为env01。 在Anaconda Prompt中使用命令：

```conda create -n env01 python=3.6 ```
### 1.3 安装GPU版本的tensorflow##
首先装好GPU的驱动(CUDA 10.0)和相关库(CUDA Toolkit)直接下载安装即可，另外一个cuDNN 7.5.1.10的安装见[Nvidia官网](https://docs.nvidia.com/deeplearning/sdk/cudnn-install/index.html#installwindows)  ，所以可以在这个基础上直接安装tensorflow的GPU版本。

进入env01环境:

```activate env01```

安装tf:

``` pip install tensorflow-gpu ```

安装好后可以查看tensorflow的版本信息（当然你也可以指定安装版本进行安装，我是采用默认的安装方式）：

``` conda list tensorflow ```

![tensorflow安装结果](tensorflow安装结果.png)


### 1.4 为Jupyter安装新的kernel ###

我自己经常使用Jupyter Notebook或者Jupyter Lab来进行实验，所以为了能够在Jupyter Lab上使用到这个新的环境配置，就需要安装新的kernel。可以参见[IPython的使用文档](https://ipython.readthedocs.io/en/latest/install/kernel_install.html)和[Jupyter安装,配置手册](https://jupyter.readthedocs.io/en/latest/projects/content-projects.html)。
```
activate env01
conda install ipykernel # or pip install ipykernel
python -m ipykernel install --user --name env01 --display-name "env01(py36)"
```
![安装kernel到Jupyter](安装kernel到Jupyter.png)

### 1.5 补充-Jupyter包需求解决方案 ###
![JupyterPackage](JupyterPackage.png)

## 2.tensrflow基础 ##
使用低级TensorFlow APIs(TensorFlow Core): 1.实验和调试都更加直接。2.使用高级api时对内部工作方式心中有数

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

### 2.1 Tensor值 ###
TensorFlow中数据的核心单元是 tensor.

1. rank:维度的个数， 标量的秩为0.

2. shape: 每一维度上数组的长度，标量的大小为[].

3. values: 使用 numpy数组来表示tensor值

### 2.2 TensorFlow核心工作流程 ###
构建计算图（tf.Graph）和运行计算图（tf.Session）

#### 2.2.1 图 ####
结点： tf.Operation,Opertion 描述消费和产生tensor的各种计算。
边： tf.sensor 代表“值”在图中流动

图中的每个操作都有一个惟一的名称。这个名称独立于Python中分配给对象的名称。张量是以生成张量的操作命名的，后面跟着一个输出索引，如上面的“add:0”所示

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0 ) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


#### 2.2.2 TensorBoard ####
用来可视化计算图
1. 保存计算图为TensorBoard summary文件
2. 打开终端 输入 tensorboard --logdir=path/to/log-directory
3. 打开浏览器输入：localhost:6006 查看 Tensorboard

In [3]:
writer = tf.summary.FileWriter('logdir/')
writer.add_graph(tf.get_default_graph())
writer.flush()

#### 2.2.3 Session ####
会话封装了TensorFlow 运行时的状态，同时运行TensorFlow的操作。如果把tf.Graph比作.py文件的话，tf.Session就类似于python解释器

In [4]:
sess = tf.Session()
print(sess.run(total))

7.0


In [5]:
# 接受任何元组或字典类型的组合形成的多个tensors输入
print(sess.run({'xxx':(a, b), 'total':total}))

{'xxx': (3.0, 4.0), 'total': 7.0}


In [6]:
# 每一次调用tf.Session.run对计算图中的任意的tf.Tensor来说其值都只有一个
vec = tf.random_uniform(shape=(3, ))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.70536757 0.02899575 0.77724373]
[0.82061243 0.9810114  0.16284251]
(array([1.990807 , 1.3547047, 1.5366379], dtype=float32), array([2.990807 , 2.3547049, 2.5366378], dtype=float32))


#### 2.2.4 Feeding ####
placeholder: 图可以参数化以接受外部输入，即占位符。

feed_dict参数可用于覆盖图中的任何张量。占位符和其他tf.Tensors之间唯一的区别是如果没有给占位符赋值，在调用tf.Session.run时会抛出一个错误。

In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x:3, y:4.5}))
print(sess.run(z, feed_dict={x: [1,3], y:[2, 4]}))

7.5
[3. 7.]


#### 2.2.5 Datasets #### 
Placeholders 适用于单次试验，tf.data更适用于流式数据

Dataset -> tf.data.Iterator -> tf.data.Iterator.get_next

The simplest way to create an Iterator is with the tf.data.Dataset.make_one_shot_iterator method. 

If the Dataset depends on stateful operations you may need to initialize the iterator before using it, as shown below:

In [8]:
my_data = [
    [0, 1,],
    [2, 3],
    [4, 5],
    [6, 7],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [9]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        print("I catch it.")
        break

[0 1]
[2 3]
[4 5]
[6 7]
I catch it.


In [10]:
r = tf.random_normal([10, 3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        print("end of data")
        break

Instructions for updating:
Colocations handled automatically by placer.
[ 1.9407614  -0.53528005  0.6724556 ]
[ 0.5993164  -1.8759983  -0.85241926]
[-1.0402809  -0.1419276   0.79817224]
[-0.5909314  -0.92287385 -1.48785   ]
[ 0.5324018  -0.13164169 -2.0933774 ]
[ 1.8553681  -0.10748121  0.41282657]
[ 1.3322107   1.0745685  -0.37091318]
[0.190562   0.49121156 0.4689252 ]
[ 1.3927369   0.8398161  -0.20138954]
[-0.953393  -1.1707511  0.7585268]
end of data


#### 层 ####
1. creating layers

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)


In [14]:
#2. initializing layers

init = tf.global_variables_initializer()
sess.run(init)

In [16]:
# 3, 执行层
print(sess.run(y, {x: [[1,2, 3], [9, 10, 11]]}))

[[0.69293284]
 [5.6562505 ]]


In [18]:
# 4.层的简写形式，不推荐，因为无法获取Layer Object,导致调试困难

### 特征列 ###
tf.feature_column.input_layer <-------- dense columns

 one-hot <--- tf.feature_column.indicator_column <------ categorical column

In [20]:

# 数据
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

# 模型
linear_model = tf.layers.Dense(units=1)

# 损失函数
y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

# 优化器
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# 初始化网络内部结点权值
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

# 迭代训练
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print("迭代次数{0}, loss值{1}".format(i, loss_value))

print(sess.run(y_pred))

迭代次数0, loss值3.5392043590545654
迭代次数1, loss值2.5282466411590576
迭代次数2, loss值1.826331377029419
迭代次数3, loss值1.3388562202453613
迭代次数4, loss值1.000179648399353
迭代次数5, loss值0.7647536396980286
迭代次数6, loss值0.60097336769104
迭代次数7, loss值0.4869091808795929
迭代次数8, loss值0.40734460949897766
迭代次数9, loss值0.3517208695411682
迭代次数10, loss值0.31271180510520935
迭代次数11, loss值0.2852337956428528
迭代次数12, loss值0.26575925946235657
迭代次数13, loss值0.2518407106399536
迭代次数14, loss值0.24177974462509155
迭代次数15, loss值0.23439781367778778
迭代次数16, loss值0.22887729108333588
迭代次数17, loss值0.22465068101882935
迭代次数18, loss值0.2213243842124939
迭代次数19, loss值0.21862497925758362
迭代次数20, loss值0.21636304259300232
迭代次数21, loss值0.21440690755844116
迭代次数22, loss值0.21266525983810425
迭代次数23, loss值0.21107476949691772
迭代次数24, loss值0.20959143340587616
迭代次数25, loss值0.20818474888801575
迭代次数26, loss值0.20683346688747406
迭代次数27, loss值0.20552292466163635
迭代次数28, loss值0.2042427659034729
迭代次数29, loss值0.20298603177070618
迭代次数30, loss值0.20174774527549744
迭代次数

### 自然语言处理 ###
nlp的四个阶段:

1. 记忆
2. 感知
3. 认知
4. 创造

人工智能第三次浪潮特点

1. 海量大数据
2. 计算能力强
3. 应用场景
![nlp的几个方面](nlp的几个方面.jpg)